# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [74]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [75]:
# Output File (CSV)
output_data_file = "Resources/city_data.csv"
city_df = pd.read_csv(output_data_file, encoding="ISO-8859-1")
city_df

,City,City ID,Latitude,Longitude,temperature,humidity,cloudiness,wind speed
0,Lompoc,5367788,34.6391,-120.4579,291.13,67,1,4.63
1,Khatanga,2022572,71.9667,102.5000,237.80,83,100,6.09
2,San Patricio,4726521,28.0170,-97.5169,283.48,76,75,4.12
3,Chokurdakh,2126123,70.6333,147.9167,238.00,84,63,2.53
4,Ilulissat,3423146,69.2167,-51.1000,269.15,80,90,6.69
...,...,...,...,...,...,...,...,...
575,Bad Waldsee,2953320,47.9167,9.7667,274.72,93,90,3.09
576,ErdÅkertes,3053253,47.6726,19.3079,273.56,80,0,3.60
577,LÃ¼deritz,3355672,-26.6481,15.1594,291.27,89,18,2.93
578,Potrero del Llano,3521207,21.0500,-97.6833,291.15,72,90,3.09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [76]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [77]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_df[["Latitude", "Longitude"]].astype(float)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values
city_df = city_df.dropna()
humidity = city_df["humidity"].astype(float)

In [78]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [79]:
narrow_df = city_df.loc[(city_df["temperature"] < 300) & (city_df["temperature"] > 295) 
                       & (city_df["wind speed"] < 5) & (city_df["cloudiness"] < 1)]
narrow_df

,City,City ID,Latitude,Longitude,temperature,humidity,cloudiness,wind speed
24,Mataura,6201424,-46.1927,168.8643,298.42,43,0,1.21
172,Bubaque,2374583,11.2833,-15.8333,298.57,70,0,4.27
228,Deniliquin,2169068,-35.5333,144.9667,296.48,39,0,4.87
239,Cerquilho,3466429,-23.1650,-47.7436,295.15,94,0,3.60
292,Saldanha,3361934,-33.0117,17.9442,296.15,64,0,1.54
309,Bulls,2192828,-40.1667,175.3833,297.80,65,0,1.34
328,GualeguaychÃº,3433658,-33.0094,-58.5172,298.15,40,0,1.79
370,MaceiÃ³,3395981,-9.6658,-35.7353,297.15,88,0,2.57
375,Makkah al Mukarramah,104515,21.4267,39.8261,295.08,71,0,1.27
428,Rafaela,3839479,-31.2503,-61.4867,298.03,40,0,0.89


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [80]:
hotel_df = narrow_df
hotel_df['Hotel Name'] = ""
hotel_df

<ipython-input-80-ba65b806580e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,City ID,Latitude,Longitude,temperature,humidity,cloudiness,wind speed,Hotel Name
24,Mataura,6201424,-46.1927,168.8643,298.42,43,0,1.21,
172,Bubaque,2374583,11.2833,-15.8333,298.57,70,0,4.27,
228,Deniliquin,2169068,-35.5333,144.9667,296.48,39,0,4.87,
239,Cerquilho,3466429,-23.1650,-47.7436,295.15,94,0,3.60,
292,Saldanha,3361934,-33.0117,17.9442,296.15,64,0,1.54,
309,Bulls,2192828,-40.1667,175.3833,297.80,65,0,1.34,
328,GualeguaychÃº,3433658,-33.0094,-58.5172,298.15,40,0,1.79,
370,MaceiÃ³,3395981,-9.6658,-35.7353,297.15,88,0,2.57,
375,Makkah al Mukarramah,104515,21.4267,39.8261,295.08,71,0,1.27,
428,Rafaela,3839479,-31.2503,-61.4867,298.03,40,0,0.89,


In [81]:
#06-Python-APIs/3/Activities/04-Ins_NearestRestr/Solved/NearestRestr.ipynb
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


params = { 
    "radius":5000,
    "type": "hotel",
    "keyword" : "hotel",
    "key": g_key,
}

In [82]:
for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try: 
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Hotel not found.")
        
    print("------------")

Retrieving Results for Index 24: Mataura.
Closest hotel in Mataura is Southland Organic Farmstay.
------------
Retrieving Results for Index 172: Bubaque.


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel in Bubaque is BOB FISHING CLUB BIJAGOS.
------------
Retrieving Results for Index 228: Deniliquin.
Closest hotel in Deniliquin is Cottages on Edward.
------------
Retrieving Results for Index 239: Cerquilho.
Closest hotel in Cerquilho is Hotel Zanata.
------------
Retrieving Results for Index 292: Saldanha.
Closest hotel in Saldanha is Hoedjiesbaai Hotel.
------------
Retrieving Results for Index 309: Bulls.
Closest hotel in Bulls is Bulls Motel & Holiday Park.
------------
Retrieving Results for Index 328: GualeguaychÃº.
Closest hotel in GualeguaychÃº is Hotel Guayrá Gualeguaychú.
------------
Retrieving Results for Index 370: MaceiÃ³.
Closest hotel in MaceiÃ³ is Jatiúca Hotel & Resort.
------------
Retrieving Results for Index 375: Makkah al Mukarramah.
Closest hotel in Makkah al Mukarramah is Pullman Zamzam Makkah.
------------
Retrieving Results for Index 428: Rafaela.
Closest hotel in Rafaela is Amérian Rafaela Hotel.
------------
Retrieving Results for Index 501: In

In [83]:
hotel_df

,City,City ID,Latitude,Longitude,temperature,humidity,cloudiness,wind speed,Hotel Name
24,Mataura,6201424,-46.1927,168.8643,298.42,43,0,1.21,Southland Organic Farmstay
172,Bubaque,2374583,11.2833,-15.8333,298.57,70,0,4.27,BOB FISHING CLUB BIJAGOS
228,Deniliquin,2169068,-35.5333,144.9667,296.48,39,0,4.87,Cottages on Edward
239,Cerquilho,3466429,-23.1650,-47.7436,295.15,94,0,3.60,Hotel Zanata
292,Saldanha,3361934,-33.0117,17.9442,296.15,64,0,1.54,Hoedjiesbaai Hotel
309,Bulls,2192828,-40.1667,175.3833,297.80,65,0,1.34,Bulls Motel & Holiday Park
328,GualeguaychÃº,3433658,-33.0094,-58.5172,298.15,40,0,1.79,Hotel Guayrá Gualeguaychú
370,MaceiÃ³,3395981,-9.6658,-35.7353,297.15,88,0,2.57,Jatiúca Hotel & Resort
375,Makkah al Mukarramah,104515,21.4267,39.8261,295.08,71,0,1.27,Pullman Zamzam Makkah
428,Rafaela,3839479,-31.2503,-61.4867,298.03,40,0,0.89,Amérian Rafaela Hotel


In [84]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [72]:
hotel_list = hotel_df["Hotel Name"].tolist()

In [73]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

SyntaxError: EOL while scanning string literal (<ipython-input-73-507e68a4814a>, line 4)